In [ ]:
import numpy as np

In [ ]:
def rnn_step_forward(x, prev_h, Wx, Wh, b):
    """
  RNN单步前向传播，使用tanh激活单元
  Inputs:
  - x: 当前时间步数据输入(N, D).
  - prev_h: 前一时间步隐藏层状态 (N, H)
  - Wx: 输入层到隐藏层连接权重(D, H)
  - Wh:隐藏层到隐藏层连接权重(H, H)
  - b: 隐藏层偏置项(H,)

  Returns 元组:
  - next_h: 下一隐藏层状态(N, H)
  - cache: 缓存
  """
    next_h, cache = None, None
    #######################################
    #                 任务：实现RNN单步前向传播                                  
    #               将输出值储存在next_h中，                                     
    #         将反向传播时所需的各项缓存存放在cache中                            
    #######################################
    a = prev_h.dot(Wh) + x.dot(Wx) + b
    next_h = np.tanh(a)
    cache = (x, prev_h, Wh, Wx, b, next_h)
    #######################################
    #                             结束编码                                       
    #######################################
    return next_h, cache

def rnn_step_backward(dnext_h, cache):
    """
  RNN单步反向传播。
  Inputs:
  - dnext_h: 后一时间片段的梯度。
  - cache: 前向传播时的缓存。
  
  Returns 元组:
  - dx: 数据梯度(N, D)。
  - dprev_h: 前一时间片段梯度(N, H)。
  - dWx: 输入层到隐藏层权重梯度(D,H)。
  - dWh:  隐藏层到隐藏层权重梯度(H, H)。
  - db: 偏置项梯度(H,)。
  """
    dx, dprev_h, dWx, dWh, db = None, None, None, None, None
    #######################################
    #              任务：实现RNN单步反向传播                                     
    #      提示：tanh(x)梯度:  1 - tanh(x)*tanh(x)                               
    #######################################

    x, prev_h, Wh, Wx, b, next_h = cache
    dscores = dnext_h * (1 - next_h * next_h)  # tanh
    dWx = np.dot(x.T, dscores)
    db = np.sum(dscores, axis=0)
    dWh = np.dot(prev_h.T, dscores)
    dx = np.dot(dscores, Wx.T)
    dprev_h = np.dot(dscores, Wh.T)

    #######################################
    #                               结束编码                                     
    #######################################
    return dx, dprev_h, dWx, dWh, db

In [ ]:
def rnn_forward(x, h0, Wx, Wh, b):
    """
  RNN前向传播。
  Inputs:
  - x: 完整的时序数据 (N, T, D)。
  - h0: 隐藏层初始化状态 (N, H)。
  - Wx: 输入层到隐藏层权重 (D, H)。
  - Wh:  隐藏层到隐藏层权重(H, H)。
  - b: 偏置项(H,)。
  
  Returns 元组:
  - h: 所有时间步隐藏层状态(N, T, H)。
  - cache: 反向传播所需的缓存。
  """
    h, cache = None, None
    #######################################
    #                     任务：实现RNN前向传播。                                
    #        提示： 使用前面实现的rnn_step_forward 函数。                        
    #######################################
    N, T, D = x.shape
    (H, ) = b.shape
    h = np.zeros((N, T, H))
    prev_h = h0
    for t in range(T):
        xt = x[:, t, :]
        next_h, _ = rnn_step_forward(xt, prev_h, Wx, Wh, b)
        prev_h = next_h
        h[:, t, :] = prev_h
    cache = (x, h0, Wh, Wx, b, h)
    #######################################
    #                           结束编码                                         #
    #######################################
    return h, cache


def rnn_backward(dh, cache):
    """
  RNN反向传播。
  Inputs:
  - dh: 隐藏层所有时间步梯度(N, T, H)。
  Returns 元组:
  - dx: 输入数据时序梯度(N, T, D)。
  - dh0: 初始隐藏层梯度(N, H)。
  - dWx: 输入层到隐藏层权重梯度(D, H)。
  - dWh: 隐藏层到隐藏层权重梯度(H, H)。
  - db: 偏置项梯度(H,)。
  """
    dx, dh0, dWx, dWh, db = None, None, None, None, None
    #######################################
    #                任务：实现RNN反向传播。                                     
    #            提示：使用 rnn_step_backward函数。                              
    #######################################
    x, h0, Wh, Wx, b, h = cache
    N, T, H = dh.shape
    _, _, D = x.shape
    
    next_h = h[:, T - 1, :]  # 最后一步
    dprev_h = np.zeros((N, H))
    
    dx = np.zeros((N, T, D))
    dh0 = np.zeros((N, H))
    dWx = np.zeros((D, H))
    dWh = np.zeros((H, H))
    db = np.zeros((H, ))
    
    for t in range(T):
        t = T - 1 - t
        xt = x[:, t, :]
        
        if t == 0:
            prev_h = h0
        else:
            prev_h = h[:, t - 1, :]  # 前一层
        
        step_cache = (xt, prev_h, Wh, Wx, b, next_h)
        next_h = prev_h  # 下次循环的前一层
        
        dnext_h = dh[:, t, :] + dprev_h  # 前一层梯度，当前层输入梯度之和
        dx[:, t, :], dprev_h, dWxt, dWht, dbt = rnn_step_backward(
            dnext_h, step_cache)
        dWx, dWh, db = dWx + dWxt, dWh + dWht, db + dbt
    
    dh0 = dprev_h
    #######################################
    #                               结束编码                                     #
    #######################################
    return dx, dh0, dWx, dWh, db

In [ ]:
def sigmoid(x):
    """
  数值稳定版本的sigmoid函数。
  """
    pos_mask = (x >= 0)
    neg_mask = (x < 0)
    z = np.zeros_like(x)
    z[pos_mask] = np.exp(-x[pos_mask])
    z[neg_mask] = np.exp(x[neg_mask])
    top = np.ones_like(x)
    top[neg_mask] = z[neg_mask]
    return top / (1 + z)


def lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b):
    """
  LSTM单步前向传播
  
  Inputs:
  - x: 输入数据 (N, D)
  - prev_h: 前一隐藏层状态 (N, H)
  - prev_c: 前一细胞状态(N, H)
  - Wx: 输入层到隐藏层权重(D, 4H)
  - Wh: 隐藏层到隐藏层权重 (H, 4H)
  - b: 偏置项(4H,)
  
  Returns 元组:
  - next_h:  下一隐藏层状态(N, H)
  - next_c:  下一细胞状态(N, H)
  - cache: 反向传播所需的缓存
  """
    next_h, next_c, cache = None, None, None
    #######################################
    #              任务：实现LSTM单步前向传播。                                 
    #         提示：稳定版本的sigmoid函数已经帮你实现，直接调用即可。           
    #               tanh函数使用np.tanh。                                       
    #######################################
    N, D = x.shape
    N, H = prev_h.shape
    
    input_gate = sigmoid(
        np.dot(x, Wx[:, 0:H]) + np.dot(prev_h, Wh[:, 0:H]) + b[0:H])
    forget_gate = sigmoid(
        np.dot(x, Wx[:, H:2 * H]) + np.dot(prev_h, Wh[:, H:2 * H]) +
        b[H:2 * H])
    output_gate = sigmoid(
        np.dot(x, Wx[:, 2 * H:3 * H]) + np.dot(prev_h, Wh[:, 2 * H:3 * H]) +
        b[2 * H:3 * H])
    
    inputs = np.tanh(
        np.dot(x, Wx[:, 3 * H:4 * H]) + np.dot(prev_h, Wh[:, 3 * H:4 * H]) +
        b[3 * H:4 * H])
    
    next_c = forget_gate * prev_c + inputs * input_gate
    next_scores_c = np.tanh(next_c)
    next_h = output_gate * next_scores_c
    cache = (x, Wx, Wh, b, inputs, input_gate, output_gate, forget_gate, prev_h,
             prev_c, next_scores_c)
    #######################################
    #                               结束编码                                     #
    #######################################

    return next_h, next_c, cache


def lstm_step_backward(dnext_h, dnext_c, cache):
    """
   LSTM单步反向传播
  
  Inputs:
  - dnext_h: 下一隐藏层梯度 (N, H)
  - dnext_c: 下一细胞梯度 (N, H)
  - cache: 前向传播缓存
  
  Returns 元组:
  - dx: 输入数据梯度 (N, D)
  - dprev_h: 前一隐藏层梯度 (N, H)
  - dprev_c: 前一细胞梯度(N, H)
  - dWx: 输入层到隐藏层梯度(D, 4H)
  - dWh:  隐藏层到隐藏层梯度(H, 4H)
  - db:  偏置梯度(4H,)
  """
    dx, dprev_h, dc, dWx, dWh, db = None, None, None, None, None, None
    #######################################
    #                           任务：实现LSTM单步反向传播                      
    #          提示：sigmoid(x)函数梯度：sigmoid(x)*(1-sigmoid(x))              
    #                tanh(x)函数梯度：   1-tanh(x)*tanh(x)                      
    #######################################
    x, Wx, Wh, b, inputs, input_gate, output_gate, forget_gate, prev_h, prev_c, next_scores_c = cache
    N, D = x.shape
    N, H = prev_h.shape
    
    dWx = np.zeros((D, 4 * H))
    dxx = np.zeros((D, 4 * H))
    dWh = np.zeros((H, 4 * H))
    dhh = np.zeros((H, 4 * H))
    db = np.zeros(4 * H)
    dx = np.zeros((N, D))
    dprev_h = np.zeros((N, H))
    
    dc_tem = dnext_c + dnext_h * (1 - next_scores_c**2) * output_gate
    dprev_c = forget_gate * dc_tem
    
    dforget_gate = prev_c * dc_tem
    dinput_gate = inputs * dc_tem
    dinput = input_gate * dc_tem
    doutput_gate = next_scores_c * dnext_h
    
    dscores_in_gate = input_gate * (1 - input_gate) * dinput_gate
    dscores_forget_gate = forget_gate * (1 - forget_gate) * dforget_gate
    dscores_out_gate = output_gate * (1 - output_gate) * doutput_gate
    dscores_in = (1 - inputs**2) * dinput
    
    da = np.hstack(
        (dscores_in_gate, dscores_forget_gate, dscores_out_gate, dscores_in))
    
    dWx = np.dot(x.T, da)
    dWh = np.dot(prev_h.T, da)
    
    db = np.sum(da, axis=0)
    dx = np.dot(da, Wx.T)
    dprev_h = np.dot(da, Wh.T)
    #######################################
    #                               结束编码                                     #
    #######################################

    return dx, dprev_h, dprev_c, dWx, dWh, db


def lstm_forward(x, h0, Wx, Wh, b):
    """
  LSTM前向传播
  Inputs:
  - x: 输入数据 (N, T, D)
  - h0:初始化隐藏层状态(N, H)
  - Wx: 输入层到隐藏层权重 (D, 4H)
  - Wh: 隐藏层到隐藏层权重(H, 4H)
  - b: 偏置项(4H,)
  
  Returns 元组:
  - h: 隐藏层所有状态 (N, T, H)
  - cache: 用于反向传播的缓存
  """
    h, cache = None, None
    #######################################
    #                    任务： 实现完整的LSTM前向传播                          
    #######################################
    N, T, D = x.shape
    H = b.shape[0] / 4
    h = np.zeros((N, T, H))
    cache = {}
    prev_h = h0
    prev_c = np.zeros((N, H))
    for t in range(T):
        xt = x[:, t, :]
        next_h, next_c, cache[t] = lstm_step_forward(xt, prev_h, prev_c, Wx,
                                                     Wh, b)
        prev_h = next_h
        prev_c = next_c
        h[:, t, :] = prev_h

    #######################################
    #                               结束编码                                     #
    #######################################

    return h, cache


def lstm_backward(dh, cache):
    """
  LSTM反向传播
  Inputs:
  - dh: 各隐藏层梯度(N, T, H)
  - cache: V前向传播缓存
  
  Returns 元组:
  - dx: 输入数据梯度 (N, T, D)
  - dh0:初始隐藏层梯度(N, H)
  - dWx: 输入层到隐藏层权重梯度 (D, 4H)
  - dWh: 隐藏层到隐藏层权重梯度 (H, 4H)
  - db: 偏置项梯度 (4H,)
  """
    dx, dh0, dWx, dWh, db = None, None, None, None, None
    #######################################
    #                 任务：实现完整的LSTM反向传播                              
    #######################################
    N, T, H = dh.shape
    x, Wx, Wh, b, input, input_gate, output_gate, forget_gate, prev_h, prev_c, next_scores_c = cache[
        T - 1]
    D = x.shape[1]
    dprev_h = np.zeros((N, H))
    dprev_c = np.zeros((N, H))
    dx = np.zeros((N, T, D))
    dh0 = np.zeros((N, H))
    dWx = np.zeros((D, 4 * H))
    dWh = np.zeros((H, 4 * H))
    db = np.zeros((4 * H, ))
    for t in range(T):
        t = T - 1 - t
        step_cache = cache[t]
        dnext_h = dh[:, t, :] + dprev_h
        dnext_c = dprev_c
        dx[:, t, :], dprev_h, dprev_c, dWxt, dWht, dbt = lstm_step_backward(
            dnext_h, dnext_c, step_cache)
        dWx, dWh, db = dWx + dWxt, dWh + dWht, db + dbt
    dh0 = dprev_h
    #######################################
    #                               结束编码                                     #
    #######################################

    return dx, dh0, dWx, dWh, db